In [11]:
import tensorflow
import numpy as np
import keras
from keras.datasets import mnist
import matplotlib.pyplot as plt
from keras.models import Sequential

In [12]:
def define_discriminator(in_shape = (28, 28, 1)):
    model = Sequential()
    model.add(keras.layers.Conv2D(40, (5, 5), strides = (1, 1), padding = 'valid'))
    model.add(keras.layers.LeakyReLU(alpha = 0.2))
    model.add(keras.layers.Dropout(0.4))
    model.add(keras.layers.Conv2D(60, (3, 3), strides = (1, 1), padding = 'valid'))
    model.add(keras.layers.LeakyReLU(alpha = 0.2))
    model.add(keras.layers.Dropout(0.4))
    model.add(keras.layers.Conv2D(80, (3, 3), strides = (1, 1), padding = 'valid'))
    model.add(keras.layers.LeakyReLU(alpha = 0.2))
    model.add(keras.layers.Dropout(0.4))
    model.add(keras.layers.Conv2D(100, (3, 3), strides = (1, 1), padding = 'valid'))
    model.add(keras.layers.LeakyReLU(alpha = 0.2))
    model.add(keras.layers.Dropout(0.4))
    model.add(keras.layers.Conv2D(120, (3, 3), strides = (1, 1), padding = 'valid'))
    model.add(keras.layers.LeakyReLU(alpha = 0.2))
    model.add(keras.layers.Dropout(0.4))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(1, activation = 'sigmoid'))
    
    model.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return model

In [13]:
def define_generator(latent_dim):
    gen_model = Sequential()
    nodes = 128 * 7 * 7
    gen_model.add(keras.layers.Dense(nodes, input_dim = latent_dim))
    gen_model.add(keras.layers.LeakyReLU(alpha = 0.2))
    gen_model.add(keras.layers.Reshape((7, 7, 128)))
    gen_model.add(keras.layers.Conv2DTranspose(128, (4, 4), strides = (2, 2), padding = 'valid'))
    gen_model.add(keras.layers.LeakyReLU(alpha = 0.2))
    
    gen_model.add(keras.layers.Conv2DTranspose(128, (4, 4), strides = (2, 2), padding = 'valid'))
    gen_model.add(keras.layers.LeakyReLU(alpha = 0.2))
    
    gen_model.add(keras.layers.Conv2D(1, (7, 7), padding = 'valid', activation = 'sigmoid'))
    
    return gen_model

In [14]:
def define_gans(g_model, d_model):
    d_model.trainable = False
    model = Sequential()
    model.add(g_model)
    model.add(d_model)
    
    model.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    return model

In [15]:
def load_real_samples():
    (trainX, _), (_, _) = mnist.load_data()
    X = np.expand_dims(trainX, axis = -1)
    X = X/255
    return X

In [16]:
def generate_real_samples(dataset, n_samples):
    idx = np.random.randint(0, dataset.shape[0], n_samples)
    X = dataset[idx]
    Y = np.ones((n_samples, 1))
    return X, Y

In [17]:
def generate_latent_points(latent_dim, n_samples):
    X = np.random.randn(latent_dim * n_samples)
    X = X.reshape(n_samples, latent_dim)
    return X

In [18]:
def generate_fake_samples(gen_model, latent_dim, n_samples):
    X_input = generate_latent_points(latent_dim, n_samples)
    p_g = gen_model.predict(X_input)
    Y = np.zeros((n_samples, 1))
    return p_g, Y

In [ ]:
def viz_plot(example, epoch, n = 10):
    for i in range(n * n):
        plt.subplot(n, n , i + 1)
        plt.imshow(example[i, :, :, :], cmap = 'gray')
    file_name = f'C:/Users/bhatt/OneDrive/Desktop/whisper-main/DS_PP_Aug_23/Machine Learning/{epoch}_epoch.jpg'
    plt.savefig(file_name)
    
    return plt

In [20]:
def architecture(g_model, d_model, dataset, latent_dim, epoch, batch = 100):
    X_real, Y_real = generate_real_samples(dataset, batch)
    
    _, acc_real = d_model.evaluate(X_real, Y_real)
    
    X_fake, Y_fake = generate_fake_samples(g_model, latent_dim, batch)
    
    _, acc_fake = d_model.evaluate(X_fake, Y_fake)
    
    print(f'Accuracy real: {acc_real}, Accuracy fake: {acc_fake}')
    
    plot = viz_plot(X_fake, epoch)

In [23]:
def train(g_model, d_model, gan_model, dataset, latent_dim, epoch, batch_size = 100):
    batch_per_epoch = int(dataset.shape[0] / batch_size)
    half_batch = int(batch_size / 2)
    
    for i in range(epoch):
        for j in range(batch_per_epoch):
            X_real, Y_real = generate_real_samples(dataset, half_batch)
            
            X_fake, Y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
            
            d_loss_real, _ = d_model.train_on_batch(X_real, Y_real)
            d_loss_fake, _ = d_model.train_on_batch(X_fake, Y_fake)
            
            d_loss = d_loss_fake + d_loss_real
            
            X_noise = generate_latent_points(latent_dim, batch_size)
            
            Y_noise = np.ones((batch_size, 1))
            
            g_loss, _ = gan_model.train_on_batch(X_noise, Y_noise)
            
            print(f'{i}, Dloss: {d_loss}, Gloss: {g_loss}')
            
        if i % 5 == 0:
            architecture(g_model, d_model, dataset, latent_dim, i)

In [ ]:
latent_dim = 100

d_model = define_discriminator()

g_model = define_generator(latent_dim)

gan_model = define_gans(g_model, d_model)

dataset = load_real_samples()

train(g_model, d_model, gan_model, dataset, latent_dim, 100)

2/2 [==============================] - 0s 41ms/step
0, Dloss: 1.9882464408874512, Gloss: 0.6105590462684631
2/2 [==============================] - 0s 54ms/step
0, Dloss: 1.3241750001907349, Gloss: 0.6906095147132874
2/2 [==============================] - 0s 44ms/step
0, Dloss: 1.3665519952774048, Gloss: 0.705746054649353
2/2 [==============================] - 0s 53ms/step
0, Dloss: 1.3629982471466064, Gloss: 0.7167156338691711
2/2 [==============================] - 0s 42ms/step
0, Dloss: 1.3348028659820557, Gloss: 0.7303139567375183
2/2 [==============================] - 0s 38ms/step
0, Dloss: 1.2439468502998352, Gloss: 0.7559817433357239
2/2 [==============================] - 0s 37ms/step
0, Dloss: 1.0224929451942444, Gloss: 0.8234013915061951
2/2 [==============================] - 0s 42ms/step
0, Dloss: 0.6306064203381538, Gloss: 1.09767484664917
2/2 [==============================] - 0s 58ms/step
0, Dloss: 0.2884739183355123, Gloss: 2.158461332321167
2/2 [===========================

2/2 [==============================] - 0s 55ms/step
0, Dloss: 1.2358958301206258e-10, Gloss: 23.49102210998535
2/2 [==============================] - 0s 33ms/step
0, Dloss: 6.81078431499782e-11, Gloss: 23.522144317626953
2/2 [==============================] - 0s 44ms/step
0, Dloss: 7.281418198425149e-11, Gloss: 23.465181350708008
2/2 [==============================] - 0s 56ms/step
0, Dloss: 6.962123889550642e-11, Gloss: 23.518842697143555
2/2 [==============================] - 0s 29ms/step
0, Dloss: 9.83174271163012e-11, Gloss: 23.483335494995117
2/2 [==============================] - 0s 29ms/step
